# Analysis of protein MD simulation data with MDTraj

[MDTraj](http://mdtraj.org) is a popular Python package for the analysis of MD simulation data. This workshop provides a brief introduction to it.

`MDTraj` integrates closely with [numpy](http://numpy.org), and a good understanding of how to use `numpy` will help you get the most out of the package, but we will not go much into that here.

`MDTraj` doesn't provide any graphical outputs, so we will use [Matplotlib](http://matplotlib.org) for that.

The data used here comes from an extended (2 nanosecond) "production" MD simulation of the Mcl-1 protein (5fdr) introduced in the previous workshops.

## 1. Import the required packages:

In [ ]:
import mdtraj as mdt
from matplotlib import pyplot as plt
import numpy as np

## 2. Load some MD simulation data

`MDTraj` works with *trajectories* - objects that encapsulate information about the behaviour of a molecular system over a period of time. 

*Trajectories* can be created by loading data from MD simulation files in a wide variety of formats (so you can use `MDTraj` with data generated by Amber, Gromacs, NAMD, Charmm, OpenMM, etc.).

Typically each *trajectory* object contains positional information (coordinates of atoms, and how these change over time), and topological information (the names of atoms, what elements they are, what residues they belong to, what other atoms they are bonded to, etc.). To obtain this data, `MDTraj` reads both MD trajectory (or single-snapshot) coordinate files, and topology files, e.g.:

In [ ]:
# A trajectory with data from a "production" simulation:
t = mdt.load('5fdr_A_amber_production.nc', top='5fdr_A.prmtop')
# A single-snapshot trajectory, corresponding to the initial structure:
tref = mdt.load('5fdr_A.inpcrd', top='5fdr_A.prmtop')

A *trajectory* object , e.g. `t`, has a number of attributes:
 - `t.topology`: the topology of the system.
 - `t.xyz`: the coordinates of each snapshot in the form of a 3-dimensional `numpy` array [*n_frames*, *n_atoms*, 3].
 - `t.time`: the timestamp for each snapshot, as a 1-dimensional `numpy` vector of length *n_frames*.
 - `t.unitcell_lengths`: the dimensions of the periodic box, as a 2-dimensional `numpy` array [*n_frames*, 3].
 - `t.unitcell_angles`: the periodic box angles, as a 2-dimensional `numpy` array [*n_frames*, 3].

Coordinates and lengths are always in **nanometers**, time in **picoseconds**, and angles in **radians**

Have a look at some of these:

In [ ]:
print(t) # A summary of the trajectory object
print(t.topology) # a summary of the topology object
print(t.xyz.shape) # the shape of the numpy array holding coordinates
print(t.time[:10]) # The values of the timestampos for the first 10 snapshots
print(t.unitcell_lengths[:5]) # The unit cell lengths in the first 5 snapshots

## 3. Select subsets of the data

Our trajectories contain information not just about the protein, but about the solvent and ions too. For some analyses, we may want to select out just certain subsets of all the atoms for analysis. For other analyses, we may only want to consider subsections of the whole trajectory by time (e.g. only the last nanosecond).`MDTraj` *trajectories* can be "sliced" in both ways.

To select just a subset of all the snapshots, the *trajectory* objects can be sliced using standard Python syntax, e.g.:
```
t_last_nanosecond = t[50:]
t_every_100_picoseconds = t[::10]
```
To select a subset of all the atoms is a two-step process. First we use the `select` method of the *topology* object to create a list of the indices of atoms that meet the criterea we want, then we use this list of atom indices with the `atom_slice` method of a *trajectory* to create a new sub-trajectory, e.g.:
```
c_alpha_atoms = t.topology.select("name CA")
t_c_alpha = t.atom_slice(c_alpha_atoms)
```

Try both of these out at once:

In [ ]:
protein_heavy_atoms = t.topology.select('protein and mass > 2.0')
t_protein_heavy = t.atom_slice(protein_heavy_atoms)
t_protein_heavy_first_nanosecond = t_protein_heavy[:50]
# What have we created:
print(t_protein_heavy_first_nanosecond)
print(t_protein_heavy_first_nanosecond.topology)

`MDTraj`'s atom selection syntax is very flexible - see [here](https://www.mdtraj.org/1.9.8.dev0/atom_selection.html) for a guide.

## 4. RMSD analysis

The first question in any MD analysis is nearly always "how much has the structure changed over the simulation from where it started?". The most common metric for this is the **root-mean-square deviation (RMSD)**. It is a pair-wise metric (i.e., a measure of the difference between two conformations of the same system), has a value of zero if the two conformations are identical, and gets larger as they get more different in shape. Its units are the same as the units of the coordinates - so for `MDTraj`, they will be *nanometers*. The value of the RMSD also depends on which atoms of the two systems are included in the analysis (you might choose all protein atoms, or maybe just protein heavy atoms, or maybe just backbone atoms, or maybe just C-alpha atoms). 

Let's calculate the RMSD of each snapshot in the production simulation from the reference (starting) structure, considering all protein atoms in the analysis. This used `MDTraj`'s [`.rmsd` method](https://www.mdtraj.org/1.9.8.dev0/api/generated/mdtraj.rmsd.html#mdtraj.rmsd). We will then plot the results using `matplotlib`:

In [ ]:
times = t.time # timestamp for each snapshot
selection = t.topology.select('protein')
# Alternatives to try later:
# selection = t.topology.select('protein and mass > 2.0') # protein heavy atoms
# selection = t.topology.select('backbone') # protein backbone atoms (N, CA, C, O)
# selection = t.topology.select('name CA') # protein C-alpha atoms
rmsd = mdt.rmsd(t, tref, atom_indices=selection) # calculate RMDSs relative to the staring structure (tref)
plt.plot(times, rmsd)
plt.xlabel('time (ps)')
plt.ylabel('RMSD (nanometers)')

Experiment with the other atom selections suggested above. Compare the RMSD curves obtained.

A naive interpretation of the RMSD data might be that after about 1 nanosecond, the conformation of the protein has stabilized around a shape about 0.21 nanometers RMSD from the starting shape - however we will discuss later why this may be misleading.

## 5. Secondary structure analysis

Another approach to considering how the conformation of the protein has changed over the MD simulation is to analyse it's *secondary structure*. For this we can use `MDTraj`'s [`.compute_dssp` method](https://www.mdtraj.org/1.9.8.dev0/api/generated/mdtraj.compute_dssp.html#mdtraj.compute_dssp). To see how it works, apply it first to the reference structure, but first cut it down to just the protein atoms:

In [ ]:
tref_protein = tref.atom_slice(tref.topology.select('protein'))
ss_ref = mdt.compute_dssp(tref_protein)
print(ss_ref)

The output is a code for each residue in the protein ('H' = helical, 'E' = beta-strand, 'C' = random coil). Now repeat, for each snapshot in the production MD simulation:

In [ ]:
t_protein = t.atom_slice(t.topology.select('protein'))
ss = mdt.compute_dssp(t_protein)
print(ss[0]) # just print the codes for the first snapshot

Because of `MDTraj`'s close relationship with `numpy`, we can now use `numpy` tricks to calculate the percentage of the amino acids in each snaphot of the production simulation that are in the same secondary structure class as they are in the initial structure:

In [ ]:
n_residues = len(ss[0])
percent_conservation = 100 * (ss == ss_ref[0]).sum(axis=1) / n_residues
plt.plot(percent_conservation)
plt.xlabel('time (ps)')
plt.ylabel('secondary structure conservation (%)')

We can see that over the whole simulation, typically around 95% of the original secondary structure is conserved.

## 6. Solvent accessible surface analysis

We recall that one of the reasons we are doing this MD simulation is to see if the hydrophobic groove in the Mcl-1 protein structure that binds bax/bak and is also the target for small molecule inhibitors is stable in their absence, since it's exposure to solvent could be energetically unfavourable.

`MDTraj` implements the [Shrake-Rupley method](https://www.mdtraj.org/1.9.8.dev0/api/generated/mdtraj.shrake_rupley.html#mdtraj.shrake_rupley) to calculate solvent accessible surface areas (SASA) for snapshots in a trajectory. To calculate the approximate contributions of polar and non-polar atoms to this, we can:

1. Strip the trajectory down to just protein heavy atoms.
2. Calculate the solvent accessible surface area of each atom in each snapshot.
3. For each snapshot, sum the SASA of all C and S atoms to get the hydrophobic component.
4. Do the same for all N and O atoms to get the hydrophilic component.

Here's the code:

In [ ]:
sasa = mdt.shrake_rupley(t_protein_heavy)
hydrophobic_atoms = t_protein_heavy.topology.select('element C or element S')
hydrophobic_sasa = sasa[:, hydrophobic_atoms].sum(axis=1)
hydrophilic_atoms = t_protein_heavy.topology.select('element N or element O')
hydrophilic_sasa = sasa[:, hydrophilic_atoms].sum(axis=1)
times = t_protein_heavy.time
plt.plot(times, hydrophobic_sasa, label='hydrophobic atoms')
plt.plot(times, hydrophilic_sasa, label='hydrophilic atoms')
plt.xlabel('time (ps)')
plt.ylabel('SASA (nm^2)')
plt.legend()

We cvan see that, as might have been predicted, as the simulation progresses there is a modest but noticeable reduction in the amount of hydrophobic surface area of the protein that is exposed to solvent, while the amount of hydrophilic surface area stays about the same.

## Summary

In this workshop we have just scratched the surface of what `MDTraj` can do. See the [examples page on the MDTraj website](https://www.mdtraj.org/1.9.8.dev0/examples/index.html) for many more ideas.